<h1> A CNN for Genre Classification </h1>

Source: https://www.youtube.com/watch?v=dOG-HxpbMSw&list=PL-wATfeyAMNrtbkCNsLcpoAyBBRJZVlnf&index=16

Data source: see my GenreClassification.ipynb notebook

In [42]:
import numpy as np
import json

In [43]:
import numpy as np
#load data

# split data into training and test

def load_data(dataset_path):
    with open(dataset_path, "r") as fp:
        data = json.load(fp)

        #convert lists into numpy arrays
        # x
        inputs=np.array(data["mfcc"])
        targets=np.array(data["labels"])
        return inputs,targets


In [44]:
inputs,targets = load_data("data.json")
#inputs.shape[0] num samples
#inputs.shape[1] num time readings
#inputs.shape[2] num values per time interval
print ("input shape", inputs.shape)

unique_elements, counts_elements = np.unique(targets, return_counts=True)
print("Frequency of unique values of the targets:")
print(np.asarray((unique_elements, counts_elements)))

input shape (4994, 293, 13)
Frequency of unique values of the targets:
[[  0   1   2   3   4   5   6   7   8   9]
 [500 500 500 500 499 495 500 500 500 500]]


In [45]:
from sklearn.model_selection import train_test_split


#create train, validation test sets

#build the cnn

#compile the network

#train the cnn

#evaluate the cnn

#infer from sample

#CNN expects 3d array of input
def prepare_datasets(test_size,validation_size,dataset_path):

    x,y = load_data(dataset_path)

    #test set: test on the fully trained model
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=test_size)
    #validation: test on unseen data, but model may end up learning it as we tweak to maximise validation accuracy
    x_train,x_validation,y_train,y_validation=train_test_split(x_train,y_train,test_size=validation_size)
    print (x_train.shape)
    x_train=x_train[...,np.newaxis]
    print (x_train.shape)
    x_validation=x_validation[...,np.newaxis]
    x_test=x_test[...,np.newaxis]
    return x_train,x_test,y_train,y_test, x_validation, y_validation
    
    
x_train,x_test,y_train,y_test, x_validation, y_validation = prepare_datasets(0.25,0.2, "data.json")

(2996, 293, 13)
(2996, 293, 13, 1)


In [46]:
#(num samples, num intervals, num variables, num channels)
x_train.shape

(2996, 293, 13, 1)

In [47]:
import tensorflow.keras as keras

def build_model(input_shape):
    model=keras.Sequential()
    #CONV LAYER 1
    model.add(keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation="relu", input_shape=input_shape))
    model.add(keras.layers.MaxPool2D(pool_size=(3,3),strides=(2,2), padding='same'))
    #Batch normalisation: process that normalises the activations in the current layer for output to the next layer.
    #Speeds up training (faster convergence) and reliability.
    model.add(keras.layers.BatchNormalization())
    
    #CONV LAYER 2
    model.add(keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation="relu", input_shape=input_shape))
    model.add(keras.layers.MaxPool2D(pool_size=(3,3),strides=(2,2), padding='same'))
    model.add(keras.layers.BatchNormalization())   
    
    #CONV LAYER 3
    # we shrink the kernal size
    model.add(keras.layers.Conv2D(filters=32,kernel_size=(2,2),activation="relu", input_shape=input_shape))
    model.add(keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2), padding='same'))
    model.add(keras.layers.BatchNormalization()) 
    
    #flatten into dense layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64,activation="relu"))
    model.add(keras.layers.Dropout(0.3))  
    
    #output layer
    #a fully connected layer for classification
    NUMBEROFPOSSIBLEOUTPUTS=10
    model.add(keras.layers.Dense(NUMBEROFPOSSIBLEOUTPUTS,activation='softmax'))
    
    return model
    
    
#each sample has the shape (n,130,13,1) <- (num samples,intervals,variables,channels)
input_shape=(x_train.shape[1], x_train.shape[2], x_train.shape[3])
model = build_model(input_shape)

model.summary()





Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 291, 11, 32)       320       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 146, 6, 32)        0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 146, 6, 32)        128       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 144, 4, 32)        9248      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 72, 2, 32)         0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 72, 2, 32)         128       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 71, 1, 32)        

In [48]:
#Adam is a very very effecting sgd variant for deep learning
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
#put all the components together
model.compile(optimizer=optimizer,
              loss="sparse_categorical_crossentropy", 
              metrics=["accuracy"]
              )



model.fit(x_train,y_train,validation_data=(x_validation,y_validation),epochs=30)

Epoch 1/30
94/94 [==============================] - 1s 6ms/step - loss: 2.4326 - accuracy: 0.2336 - val_loss: 2.2701 - val_accuracy: 0.2590
Epoch 2/30
94/94 [==============================] - 0s 5ms/step - loss: 1.8915 - accuracy: 0.3635 - val_loss: 1.7830 - val_accuracy: 0.3632
Epoch 3/30
94/94 [==============================] - 0s 5ms/step - loss: 1.6543 - accuracy: 0.4396 - val_loss: 1.5383 - val_accuracy: 0.4633
Epoch 4/30
94/94 [==============================] - 0s 5ms/step - loss: 1.5401 - accuracy: 0.4720 - val_loss: 1.4181 - val_accuracy: 0.5140
Epoch 5/30
94/94 [==============================] - 0s 5ms/step - loss: 1.4023 - accuracy: 0.5063 - val_loss: 1.3426 - val_accuracy: 0.5274
Epoch 6/30
94/94 [==============================] - 0s 5ms/step - loss: 1.3075 - accuracy: 0.5471 - val_loss: 1.3060 - val_accuracy: 0.5381
Epoch 7/30
94/94 [==============================] - 0s 5ms/step - loss: 1.2466 - accuracy: 0.5741 - val_loss: 1.2447 - val_accuracy: 0.5554
Epoch 8/30
94/94 [==

In [50]:
#evaluate CNN on test set
test_error,test_Accuracy=model.evaluate(x_test,y_test)
print ("accuracy on test set is ", test_Accuracy)

40/40 [==============================] - 0s 2ms/step - loss: 1.0038 - accuracy: 0.6501
accuracy on test set is  0.6501200795173645
